# Limpeza e preparação
- Tratamento dos dados em branco (remover, usar médias, zerar)
- Remover duplicidades
- Trasformações (dê-para). switch replace
- Renomear indices e colunas
- Discretização de colunas numéricas
- Tratar os outliers
- Converção de tipos mais adequados
- Filtragem
- Agrupamentos

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# configurações de exibição
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Tratamento de dados em branco

### Series

In [3]:
# criando uma serie com um elemento nan
strings_serie = pd.Series(["Leandro", "Barbieri", np.nan, "Moraes"])
strings_serie

0     Leandro
1    Barbieri
2         NaN
3      Moraes
dtype: object

In [4]:
# filtrando um elemento nulo, nan
strings_serie.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
# removendo elemento nulo nan
strings_serie.dropna()

0     Leandro
1    Barbieri
3      Moraes
dtype: object

In [6]:
# Usando indexação boleana. O comando acima é igual ao comando abaixo
strings_serie[strings_serie.notnull()]

0     Leandro
1    Barbieri
3      Moraes
dtype: object

### DataFrame

In [7]:
dados1 = pd.DataFrame([ [1, 2, 3], [np.nan, np.nan, np.nan], [np.nan, 4, 5] ])
dados1

,0,1,2
0,1.0,2.0,3.0
1,NaN,NaN,NaN
2,NaN,4.0,5.0


#### Removendo dropna

In [8]:
# removendo linhas (padrão): Remove todas as linhas que tenham pelo menos um NaN
dados1.dropna(axis=0)

,0,1,2
0,1.0,2.0,3.0


In [9]:
# descarta apenas as linhas que tenham todas as colunas com NaN
dados1.dropna(how="all")

,0,1,2
0,1.0,2.0,3.0
2,NaN,4.0,5.0


In [10]:
# faz com que todas as linhas da coluna 2 tenham valor nulo para testes
dados1.loc[:, 2] = np.nan

# descartar apenas as colunas (axis=1) que tenham todas as linhas (how="all") com o valor NaN
dados1.dropna(axis=1, how="all")

,0,1
0,1.0,2.0
1,NaN,NaN
2,NaN,4.0


In [11]:
# removendo com limite de quantidade
dados2 = pd.DataFrame(np.random.randn(7,3))
dados2

,0,1,2
0,-0.204708,0.478943,-0.519439
1,-0.555730,1.965781,1.393406
2,0.092908,0.281746,0.769023
3,1.246435,1.007189,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [12]:
# setando alguns valores nulos para testes
# quatro primeiras linhas da segunda coluna
dados2.iloc[:4, 1] = np.nan
# duas primeiras linhas da terceira coluna
dados2.iloc[:2, 2] = np.nan

dados2

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [13]:
# remove apenas as duas primeiras linhas apesar de existir 4 linhas com valor NaN
dados2.dropna(axis=0, thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


#### Preenchimento fillna

In [14]:
# preenche com o valor zero os dados np.nan
dados2.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [15]:
# definindo nomes para as colunas
dados2.columns = ["Primeira", "Segunda", "Terceira"]

# passa um dict com o nome de cada coluna e o valor padrão para ser usado caso seja encontrado um valor np.nan
dados2.fillna({"Primeira": 0, "Segunda": 888, "Terceira": 999})

,Primeira,Segunda,Terceira
0,-0.204708,888.000000,999.000000
1,-0.555730,888.000000,999.000000
2,0.092908,888.000000,0.769023
3,1.246435,888.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [16]:
# fillna retorna um novo df alterado, para alterar o df original
# usar inplace=True
dados2.fillna({"Primeira": 0, "Segunda": 888, "Terceira": 999}, inplace=True)
dados2

,Primeira,Segunda,Terceira
0,-0.204708,888.000000,999.000000
1,-0.555730,888.000000,999.000000
2,0.092908,888.000000,0.769023
3,1.246435,888.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


#### Interpolação ffill

In [17]:
# metodos de interpolação, preencher para frente ou para trás
dados3 = pd.DataFrame(np.random.randn(6,3), columns=["Pri", "Seg", "Ter"])
dados3

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.000940,1.343810
3,-0.713544,-0.831154,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512


In [18]:
# criando alguns dados em np.nan para testes
dados3.iloc[2:, 1] = np.nan
dados3.iloc[4:, 2] = np.nan
dados3

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [19]:
# preenche com o ultimo dado as linhas na sequencia para baixo
# limit=2 limita o preenchimento apenas nas proximas 2 linhas apos o primeiro NAN
dados3.fillna(method="ffill", limit=2)

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [20]:
# preencher com a média da coluna
dados3.fillna(dados3.mean())

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,1.686533,1.343810
3,-0.713544,1.686533,-2.370232
4,-1.860761,1.686533,-0.436259
5,-1.265934,1.686533,-0.436259


## Trasnformação dos dados

#### Removendo duplicados

In [21]:
# criando um dataframe de exemplo
dados4 = pd.DataFrame({"k1": ["um","dois"] * 3 + ["dois"], "k2": [1, 2, 1, 2, 1, 2, 2]})
dados4

,k1,k2
0,um,1
1,dois,2
2,um,1
3,dois,2
4,um,1
5,dois,2
6,dois,2


In [22]:
dados4.duplicated()

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

In [23]:
# mantém apenas os não duplicados, apaga a partir da segunda ocorrência
dados4.drop_duplicates()

,k1,k2
0,um,1
1,dois,2


In [24]:
# podemos detectar as duplicidades usando um subconjunto de colunas
# usando a coluna k1 e preservando apenas a ultima ocorrencia (o ultimo 1 da lista e o ultimo 2 da lista)
dados4.drop_duplicates(["k1"], keep="last")

,k1,k2
4,um,1
6,dois,2


#### Dê-Para

In [25]:
# DataFrame com uma coluna de comidas e um valor total
dados5 = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                      'total': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
dados5

,food,total
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [26]:
# Dicionário que será usado para fazer o de-para entre a comida e o tipo de carne
comida_carne = {
    'bacon': 'pig', 
    'pulled pork': 'pig', 
    'pastrami': 'cow', 
    'corned beef': 'cow', 
    'honey ham': 'pig', 
    'nova lox': 'salmon'
}

In [27]:
# criar uma coluna com o nome da carne de cada animal
lista_comidas = dados5["food"].str.lower()

# mapeia cada comida com as chaves do dicionario
dados5["tipo_carne"] = lista_comidas.map(comida_carne)

dados5

,food,total,tipo_carne
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [28]:
# outra forma de fazer o mapeamento entre os valores da coluna com os valores do dict
dados5["food"].map(lambda x: comida_carne[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [29]:
# Altera um valor por outro de-para
# fillna é um caso específico de substituição
dados4.replace({"um": "Primeiro", "dois": "Segundo"}, inplace=True)
dados4.replace({1: "1ª", 2: "2º"}, inplace=True)

dados4

,k1,k2
0,Primeiro,1ª
1,Segundo,2º
2,Primeiro,1ª
3,Segundo,2º
4,Primeiro,1ª
5,Segundo,2º
6,Segundo,2º


#### Renomeando indices

In [30]:
# alterando o índice das linhas
dados4.index = dados4.index.map(lambda x: x ** 2)
dados4

,k1,k2
0,Primeiro,1ª
1,Segundo,2º
4,Primeiro,1ª
9,Segundo,2º
16,Primeiro,1ª
25,Segundo,2º
36,Segundo,2º


In [31]:
# alterando nome das colunas
dados4.rename(columns=str.upper)

,K1,K2
0,Primeiro,1ª
1,Segundo,2º
4,Primeiro,1ª
9,Segundo,2º
16,Primeiro,1ª
25,Segundo,2º
36,Segundo,2º


In [32]:
# alterando nome das colunas usando um dict
dados4.rename(columns={"k1": "COLUNA1", "k2": "COLUNA2"})

,COLUNA1,COLUNA2
0,Primeiro,1ª
1,Segundo,2º
4,Primeiro,1ª
9,Segundo,2º
16,Primeiro,1ª
25,Segundo,2º
36,Segundo,2º


#### Discretização bins

In [40]:
idades = np.random.randint(0, 100, size=100)
idades.sort()
idades

array([ 2,  2,  3,  4,  5,  6,  9, 11, 11, 12, 12, 13, 13, 13, 14, 14, 16,
       17, 17, 18, 19, 19, 20, 21, 21, 23, 24, 24, 25, 28, 29, 30, 31, 31,
       32, 33, 33, 34, 36, 36, 37, 38, 38, 38, 40, 41, 41, 41, 41, 42, 43,
       43, 44, 45, 46, 47, 47, 49, 50, 50, 51, 53, 55, 55, 57, 57, 57, 59,
       60, 61, 61, 61, 62, 65, 66, 72, 75, 77, 77, 79, 79, 82, 83, 83, 83,
       85, 86, 86, 88, 88, 92, 93, 93, 94, 95, 95, 96, 96, 98, 99])

In [49]:
# definindo as faixas
bins = [0, 18, 35, 65, 100]

# Compartimentos criados: [(0, 18] < (18, 35] < (35, 65] < (65, 100]]
# rigth controla o lado do compartimento que é incluído
categorias = pd.cut(idades, bins=bins, right=True, 
                    labels=["Faixa de 0 a 18 anos", "Faixa de 19 a 35 anos", "Faixa de 36 a 65 anos", "Faixa de 66 a 100 anos"])

# parenteses representa que está incluído no range
categorias

['Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', ..., 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos']
Length: 100
Categories (4, object): ['Faixa de 0 a 18 anos' < 'Faixa de 19 a 35 anos' < 'Faixa de 36 a 65 anos' <
                         'Faixa de 66 a 100 anos']

In [50]:
# codigo de cada categoria
categorias.codes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int8)

In [51]:
categorias.categories

Index(['Faixa de 0 a 18 anos', 'Faixa de 19 a 35 anos',
       'Faixa de 36 a 65 anos', 'Faixa de 66 a 100 anos'],
      dtype='object')

In [52]:
# quantidade por catedgoria
categorias.value_counts()

Faixa de 0 a 18 anos      20
Faixa de 19 a 35 anos     18
Faixa de 36 a 65 anos     36
Faixa de 66 a 100 anos    26
dtype: int64

In [56]:
# criando compartimentos com quantidades de itens iguais, passar um valor inteiro que representa a quantidade de compartimentos
# divide a distribuição

# neste caso separa em quatro partes (quartis) com 25% dos elementos
categorias_quartis = pd.qcut(idades, 4)
categorias_quartis.value_counts()

(1.999, 22.5]    25
(22.5, 42.5]     25
(42.5, 67.5]     25
(67.5, 99.0]     25
dtype: int64

## Filtrando outliers

In [58]:
# dados de testes
dados5 = pd.DataFrame(np.random.randn(1000, 4), columns=["Um", "Dois", "Tres", "Quatro"])
dados5

,Um,Dois,Tres,Quatro
0,0.332883,-0.156736,2.010390,-0.887104
1,-0.977936,-0.267217,0.483338,-0.400333
2,0.449880,0.399594,-0.151575,-2.557934
3,0.160807,0.076525,-0.297204,-1.294274
4,-0.885180,-0.187497,-0.493560,-0.115413
...,...,...,...,...
995,0.720790,-1.721969,1.321976,0.859364
996,-0.341054,0.386810,-0.350602,-0.751669
997,-0.299946,0.236063,-0.730747,-0.393286
998,-0.671839,-0.424880,-1.129289,-0.965977


In [60]:
# exibe estatíticas que ajudam a identificar se existem dados muito fora da distribuição
# por exemplo muito acima da média, ou abaixo
dados5.describe().T

,count,mean,std,min,25%,50%,75%,max
Um,1000.0,0.060462,0.992044,-3.548824,-0.598600,0.090161,0.756646,2.653656
Dois,1000.0,0.012372,1.009703,-3.184377,-0.633502,-0.023344,0.665410,3.525865
Tres,1000.0,0.019792,0.994527,-3.745356,-0.630267,-0.007050,0.680673,3.927528
Quatro,1000.0,-0.049567,0.994312,-3.428254,-0.752280,-0.100783,0.622384,3.366626


In [63]:
# Identifica as colunas que tem valores nulos
dados5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Um      1000 non-null   float64
 1   Dois    1000 non-null   float64
 2   Tres    1000 non-null   float64
 3   Quatro  1000 non-null   float64
dtypes: float64(4)
memory usage: 31.4 KB


In [67]:
# filtrando valores acima de 3 e abaixo de -3 (corte)
dados5[np.abs(dados5) < 3].describe().T


,count,mean,std,min,25%,50%,75%,max
Um,999.0,0.064075,0.985937,-2.989741,-0.596924,0.092695,0.757111,2.653656
Dois,995.0,0.008693,0.985774,-2.925113,-0.625288,-0.023790,0.653752,2.954439
Tres,997.0,0.023079,0.975064,-2.881858,-0.622109,-0.006248,0.680397,2.735527
Quatro,998.0,-0.049604,0.983607,-2.969411,-0.751047,-0.100783,0.621519,2.666744


In [70]:
# somente os registros que estão acima de 3 e abaixo de 3. São as linhas removidas na filtragem anterior
dados5[(np.abs(dados5) > 3).any(1)]

,Um,Dois,Tres,Quatro
32,0.552936,0.106061,3.927528,-0.255126
37,-0.565230,3.176873,0.959533,-0.975340
240,0.457246,-0.025907,-3.399312,-0.974657
259,1.951312,3.260383,0.963301,1.201206
335,0.508391,-0.196713,-3.745356,-1.520113
434,-0.242459,-3.056990,1.918403,-0.578828
457,0.682841,0.326045,0.425384,-3.428254
521,1.179227,-3.184377,1.369891,-1.074833
743,-3.548824,1.553205,-2.186301,1.277104
834,-0.578093,0.193299,1.397822,3.366626


,Um,Dois,Tres,Quatro
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.067011,0.023650,0.032010,-0.043505
std,0.989807,1.005707,0.987132,0.992032
min,-2.989741,-2.925113,-2.881858,-2.969411
25%,-0.596286,-0.618832,-0.621967,-0.749803
50%,0.094503,-0.016127,-0.002014,-0.100609
75%,0.759106,0.671128,0.694484,0.623142
max,3.000000,3.000000,3.000000,3.000000


### Amostra aleatória

In [ ]:
# reordenar aleatoriamente as linhas de um df
dados6 = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
dados6